# Named Entity Recognition

In this notebook, we'll implement a simple named entity recognition (NER) tagger model and evaluate it on the [CoNLL 2003 shared task](http://www.cnts.ua.ac.be/conll2003/ner/).

In [1]:
import os, sys, re, json, time
import itertools
import collections
from IPython.display import display

# NLTK for NLP utils and corpora
import nltk

# NumPy and TensorFlow
import numpy as np
import tensorflow as tf

# Pandas because pandas are awesome, and for pretty-printing
import pandas as pd
# Set pandas floating point display
pd.set_option('float_format', lambda f: "{0:.04f}".format(f))

# Helper libraries for this notebook
import utils
reload(utils)
import vocabulary
reload(vocabulary)

<module 'vocabulary' from 'vocabulary.pyc'>

## Window-based Tagger Model

For our model, we'll implement a simple window-based model that predicts an entity tag based on the center word and its immediate neighbors.

We'll use a two-layer neural network to do this. It's based on the "Window approach network" in Figure 1 of [Collobert et al. 2011](https://arxiv.org/pdf/1103.0398v1.pdf), and also quite similar to the NPLM model that [we studied in Week 4](../week4/Neural%20Probabilistic%20Language%20Model.ipynb).

Here's a quick sketch, for a window size of $C = 3$:  
![windowtagger.png](windowtagger.png)

As in week 4, our model will have three parts. Let $k = (C-1)/2$:

- **Embedding layer:** $ f^{(i')} = W_{in}[w^{(i')}] $ for each window element $i' \in \{i-k,\ldots, i, \ldots, i+k\}$, and concatenated embeddings $x^{(i)} = [f^{(i-k)},\ldots,f^{(i+k)}]$
- **Hidden layer:** $ h^{(i)} = \tanh(x^{(i)} W_h + b_h) $
- **Output layer:** $\hat{P}(y^{(i)}) = \text{softmax}(h^{(i)} W_{out} + b_{out}) $

Note that as in Week 4 and Assignment 1, we write left multiplication to be more consistent with the TensorFlow implementation.

Our model hyperparameters are:
- `V`: vocabulary size
- `M`: embedding size
- `C`: window size
- `H`: hidden size
- `num_classes`: number of output classes

For CoNLL 2003, `num_classes = 5`: the entity types `PER`, `ORG`, `LOC`, and `MISC`, and the "other" tag `O`.

In [2]:
tf.reset_default_graph()
tf.set_random_seed(42)

##
# Hyperparameters
V = 30000
M = 50
C = 5
H = 100
num_classes = 5

k = int((C - 1) / 2)
assert(C == 2*k + 1)

with tf.name_scope("Inputs"):
  w_ = tf.placeholder(tf.int32, shape=[None, C], name="w")
  y_ = tf.placeholder(tf.int32, shape=[None], name="y")

with tf.variable_scope("Embedding_Layer"):
  W_in_ = tf.get_variable("W_in", shape=[V,M], dtype=tf.float32, 
                          initializer=tf.random_uniform_initializer(-1, 1))
  f_ = tf.nn.embedding_lookup(W_in_, w_, name="f")
  # Reshape as a simple way to concatenate embeddings
  x_ = tf.reshape(f_, [-1, C*M], name="x")
  
with tf.variable_scope("Hidden_Layer"):
  W_h_ = tf.get_variable("W_h", shape=[C*M, H], dtype=tf.float32, 
                         initializer=tf.contrib.layers.xavier_initializer())
  b_h_ = tf.get_variable("b_h", dtype=tf.float32, 
                         initializer=tf.zeros_initializer([H]))
  h_ = tf.tanh(tf.matmul(x_, W_h_) + b_h_, name="h")
  
with tf.variable_scope("Output_Layer"):
  W_out_ = tf.get_variable("W_out", shape=[H, num_classes], dtype=tf.float32, 
                         initializer=tf.contrib.layers.xavier_initializer())
  b_out_ = tf.get_variable("b_out", dtype=tf.float32, 
                           initializer=tf.zeros_initializer([num_classes]))
  logits_ = tf.add(tf.matmul(h_, W_out_), b_out_, name="logits")

with tf.name_scope("Loss_Function"):
  point_loss_ = tf.nn.sparse_softmax_cross_entropy_with_logits(logits_, y_)
  loss_ = tf.reduce_sum(point_loss_)

with tf.name_scope("Training"):
  alpha_ = tf.placeholder(tf.float32, name="learning_rate")
  optimizer_ = tf.train.AdagradOptimizer(alpha_)
  train_step_ = optimizer_.minimize(loss_)

with tf.name_scope("Prediction"):
  pred_proba_ = tf.nn.softmax(logits_, name="pred_proba")
  pred_max_ = tf.argmax(logits_, 1, name="pred_max")
    
# Initializer step
init_ = tf.initialize_all_variables()

As usual, we can inspect our model in TensorBoard. Run the cell below, then in a separate terminal, run:
```
tensorboard --logdir="~/w266/week11/tf_summaries" --port 6006
```
and go to http://localhost:6006/#graphs

It should look something like this:
![graph](graph.png)

In [3]:
summary_writer = tf.train.SummaryWriter("tf_summaries", 
                                        tf.get_default_graph())

## Loading the Dataset

We've provided a processed version of the English part of the [CoNLL 2003 dataset](http://www.cnts.ua.ac.be/conll2003/ner/), located in `data/ner/train` and `data/ner/dev`. As with many NLP datasets, the text is from newswire articles - in this case, a portion of the [Reuters corpus](http://trec.nist.gov/data/reuters/reuters.html). The training set consists of about 200k words with 30k entities.

The format looks like this (eew, tabs):
```
-DOCSTART-	O

EU	ORG
rejects	O
German	MISC
call	O
to	O
boycott	O
British	MISC
lamb	O
.	O
```
In a real model, we need to use BIO or similar encoding to handle adjacent entities properly; for this example, however, we'll just predict the raw rags.

We've provided some code in `utils.py` to read this; take a look at that file for the details.

In [4]:
tags = ["O", "PER", "ORG", "LOC", "MISC"]
tag_to_num = {t:i for i,t in enumerate(tags)}
num_to_tag = {i:t for i,t in enumerate(tags)}
print tag_to_num

# Load to list( list( (word, tag) ) )
train_docs = utils.load_dataset("data/ner/train")
dev_docs = utils.load_dataset("data/ner/dev")

# Build vocabulary
word_stream = utils.canonicalize_words(utils.flatten([w for w,t in doc] 
                                                     for doc in train_docs))
vocab = vocabulary.Vocabulary(word_stream, size=V)

# Load to windows
train_w, train_y = utils.docs_to_windows(train_docs, tag_to_num, vocab, k)
dev_w, dev_y = utils.docs_to_windows(dev_docs, tag_to_num, vocab, k)

print "Train set: %d rows with" % len(train_w),
print "%d non-O tags" % sum(train_y != tag_to_num["O"])
print "Dev set: %d rows with" % len(dev_w),
print "%d non-O tags" % sum(dev_y != tag_to_num["O"])

{'ORG': 2, 'MISC': 4, 'PER': 1, 'O': 0, 'LOC': 3}
Train set: 203621 rows with 34043 non-O tags
Dev set: 51362 rows with 8603 non-O tags


Here's a sample of the loaded data, converting the indices back to strings:

In [5]:
cols = ["w_{i-2}", "w_{i-1}", "w_{i}", "w_{i+1}", "w_{i+2}", "y"]
utils.pretty_print_matrix(np.hstack([train_w[:5], 
                                     train_y[:5].reshape([-1,1])]), 
                          cols=cols, dtype=int)

def show_window(w, y):
  return ("[ %s ]" % " ".join(vocab.ids_to_words(w)) 
          + "- %s" % num_to_tag[y])
  
for i in range(5):
  print show_window(train_w[i], train_y[i])

,w_{i-2},w_{i-1},w_{i},w_{i+1},w_{i+2},y
0,0,0,948,12151,198,2
1,0,948,12151,198,590,0
2,948,12151,198,590,10,4
3,12151,198,590,10,3989,0
4,198,590,10,3989,207,0


[ <s> <s> eu rejects german ]- ORG
[ <s> eu rejects german call ]- O
[ eu rejects german call to ]- MISC
[ rejects german call to boycott ]- O
[ german call to boycott british ]- O


## Training our Model

With our data in array form, we can train our model much like any machine learning model. The code below is almost identical to the Week 4 NPLM notebook.

We'll optimize cross-entropy loss as usual, but instead of measuring perplexity to evaluate performance, we'll measure F1 score on the non-O classes.

In [6]:
##
# Helper functions for training, to reduce boilerplate code

def train_batch(session, batch, alpha):
    feed_dict = {w_:batch[0],
                 y_:batch[1],
                 alpha_:alpha}
    c, _ = session.run([loss_, train_step_],
                       feed_dict=feed_dict)
    return c

def score_batch(session, batch):
  feed_dict = {w_:batch[0],
               y_:batch[1]}
  return session.run([loss_, pred_max_], feed_dict=feed_dict)
  
def batch_generator(w, y, batch_size):
    """Generate minibatches from data."""
    assert(len(w) == len(y))
    for i in xrange(0, len(w), batch_size):
        yield w[i:i+batch_size], y[i:i+batch_size]

We'll also define some helpers to compute and measure our predictions:

In [7]:
##
# Evaluation code for multi-class prediction
from sklearn import metrics

def eval_performance(y_true, y_pred, tagnames):
  pre, rec, f1, support = metrics.precision_recall_fscore_support(y_true, y_pred)
  avg_pre = (100*sum(pre[1:] * support[1:])/sum(support[1:]))
  avg_rec = (100*sum(rec[1:] * support[1:])/sum(support[1:]))
  avg_f1 = (100*sum(f1[1:] * support[1:])/sum(support[1:]))
  print "mean P, R, F1: %.02f%% / %.02f%% / %.02f%%" % (avg_pre, avg_rec, avg_f1)
    
def pred_dataset(session, w, y):
  y_pred = []
  for batch in batch_generator(w, y, batch_size=1000):
    _, yp = score_batch(session, batch)
    y_pred.extend(yp)
  return y_pred

And finally, we'll train our model and evaluate on the dev set! The model should train very quickly, less than a minute or so with the default parameters - although this depends somewhat on your hardware.

In [8]:
# One epoch = one pass through the training data
num_epochs = 3
batch_size = 100
alpha = 0.1  # learning rate
print_every = 1000

np.random.seed(42)

session = tf.Session()
session.run(init_)

t0 = time.time()
for epoch in xrange(1,num_epochs+1):
  t0_epoch = time.time()
  epoch_cost = 0.0
  print ""
  for i, batch in enumerate(batch_generator(train_w, train_y, batch_size)):
      if (i % print_every == 0):
          print "[epoch %d] seen %d minibatches" % (epoch, i)

      epoch_cost += train_batch(session, batch, alpha)

  avg_cost = epoch_cost / len(train_w)
  print "[epoch %d] Completed %d minibatches in %s" % (epoch, i, utils.pretty_timedelta(since=t0_epoch))
  print "[epoch %d] Average cost: %.03f" % (epoch, avg_cost,)

  ##
  # Evaluate on train and dev
  print "Train:",
  train_pred = pred_dataset(session, train_w, train_y)
  eval_performance(train_y, train_pred, tags)
  print "Dev:  ",
  dev_pred = pred_dataset(session, dev_w, dev_y)
  eval_performance(dev_y, dev_pred, tags)
  
##
# Full report on dev set
print ""
print "Dev set classification report:"
dev_pred = pred_dataset(session, dev_w, dev_y)
print metrics.classification_report(dev_y, dev_pred,
                                    target_names=tags)


[epoch 1] seen 0 minibatches
[epoch 1] seen 1000 minibatches
[epoch 1] seen 2000 minibatches
[epoch 1] Completed 2036 minibatches in 0:00:01
[epoch 1] Average cost: 0.256
Train: mean P, R, F1: 89.56% / 82.33% / 85.55%
Dev:   mean P, R, F1: 85.31% / 65.88% / 74.07%

[epoch 2] seen 0 minibatches
[epoch 2] seen 1000 minibatches
[epoch 2] seen 2000 minibatches
[epoch 2] Completed 2036 minibatches in 0:00:01
[epoch 2] Average cost: 0.094
Train: mean P, R, F1: 95.56% / 93.17% / 94.34%
Dev:   mean P, R, F1: 87.84% / 69.71% / 77.29%

[epoch 3] seen 0 minibatches
[epoch 3] seen 1000 minibatches
[epoch 3] seen 2000 minibatches
[epoch 3] Completed 2036 minibatches in 0:00:01
[epoch 3] Average cost: 0.047
Train: mean P, R, F1: 97.70% / 96.90% / 97.30%
Dev:   mean P, R, F1: 87.93% / 71.53% / 78.38%

Dev set classification report:
             precision    recall  f1-score   support

          O       0.96      0.99      0.97     42759
        PER       0.96      0.64      0.77      3149
        OR

Precision, recall, and F1 are defined as usual; "support" is just the total number of targets of that class (i.e. our dev set has 3149 words tagged as `PER` out of 51362 total).

### Exercises (not graded)

**1.** What are the dimensions of $W_h$ in this model, if $C = 3$, $M = 100$, and $H = 100$?

**2.** Would this model benefit from pre-trained word embeddings? Why or why not?

**3.** The training set has only about 24k unique words. If you use pre-trained word embeddings with a vocabulary of 300,000, should you allow the embeddings to change while training the NER tagger, or hold them fixed? How might your choice affect the ability of your model to generalize?

**4.** For simplicity, this demo maps all words to lowercase (see `utils.canonicalize_word`). Why might this be a bad idea for named entity recognition?

**5.** Does the model above overfit or underfit? How might you adjust it to improve performance?

**6.** `ORG` seems to be the lowest-performing class, based on the report above. Look at the misclassified examples - can you guess why the model might be having a hard time with this class? What features or modifications of the model might improve performance here?

In [9]:
# Get indices of misclassified "ORG" targets
mask = (dev_y == tag_to_num['ORG']) & (dev_y != dev_pred)
idxs = np.arange(len(dev_y))[mask]
for i in idxs[:20]:
  print show_window(dev_w[i], dev_y[i]),
  print "  (pred: %s)" % num_to_tag[dev_pred[i]]

[ victory while kent made up ]- ORG   (pred: PER)
[ after bowling somerset out for ]- ORG   (pred: O)
[ oval , surrey captain chris ]- ORG   (pred: LOC)
[ <s> <s> derbyshire kept up ]- ORG   (pred: PER)
[ , took derbyshire to DGDGDG ]- ORG   (pred: O)
[ to <unk> nottinghamshire for DGDGDG ]- ORG   (pred: O)
[ DGDGDG-DG , derbyshire DGDGDG ( ]- ORG   (pred: PER)
[ , the test and county ]- ORG   (pred: O)
[ the test and county cricket ]- ORG   (pred: O)
[ another against british universities , ]- ORG   (pred: MISC)
[ against the minor counties and ]- ORG   (pred: MISC)
[ the minor counties and </s> ]- ORG   (pred: O)
[ DGDG v <unk> of <unk> ]- ORG   (pred: O)
[ v <unk> of <unk> 's ]- ORG   (pred: O)
[ gloucestershire or sussex or surrey ]- ORG   (pred: O)
[ soccer - rotor fans locked ]- ORG   (pred: LOC)
[ stones at dynamo moscow players ]- ORG   (pred: LOC)
[ at dynamo moscow players during ]- ORG   (pred: LOC)
[ friday that rotor would play ]- ORG   (pred: O)
[ would play lada togliatt